In [ ]:
import os
import re
import json
import urllib.request
import ipykernel

# Try importing for both Notebook 6.x and Jupyter Server 2.x+
try:
    from notebook import notebookapp as serverapp
except ImportError:
    from jupyter_server import serverapp

def get_notebook_dir():
    # Try to read the connection file name; may be absent or oddly named
    try:
        cf = os.path.basename(ipykernel.get_connection_file())
    except Exception:
        cf = ""

    # Extract kernel id if it matches the usual pattern; otherwise None
    m = re.match(r"^kernel-(.+)\.json$", cf)
    kid = m.group(1) if m else None

    # Query any running servers for a matching session
    for s in serverapp.list_running_servers():
        api = s.get("url", "") + "api/sessions"
        try:
            with urllib.request.urlopen(api) as resp:
                sessions = json.load(resp)
            for sess in sessions:
                sess_kid = (sess.get("kernel") or {}).get("id")
                # If we couldn't parse our kernel id, accept the first session
                if kid is None or sess_kid == kid:
                    # notebook 6 uses 'notebook_dir'; jupyter_server uses 'root_dir'
                    root = s.get("notebook_dir") or s.get("root_dir") or ""
                    # session key can be 'notebook':{'path':...} or sometimes 'file':{'path':...}
                    nb_path = (sess.get("notebook") or {}).get("path") or (sess.get("file") or {}).get("path")
                    if root and nb_path:
                        return os.path.dirname(os.path.join(root, nb_path))
        except Exception:
            # Ignore servers we can't reach / auth issues, keep trying others
            pass

    # Fallback — in headless runs (papermill) this is often what you want
    return os.getcwd()

In [ ]:
from pathlib import Path
import os

TRUNCATE_AFTER = "ADS-B-BaseStationReader"

def get_project_root_folder():
    # If we're running from the command line, the project root path will be set
    path = os.environ["PROJECT_ROOT"] if "PROJECT_ROOT" in os.environ else None
    if path and os.path.exists(path):
        return path

    # Get the notebook folder and create a Path object from it
    notebook_folder = get_notebook_dir()
    path = Path(notebook_folder)

    # Find the part up to and including the final element after which we truncate
    for parent in path.parents:
        if parent.name == TRUNCATE_AFTER:
            return parent

    # Fallback - just return CWD
    return os.getcwd()



In [ ]:
from pathlib import Path

def get_data_folder_path():
    # Construct the path to the export folder and create it
    project_root = get_project_root_folder()
    data_folder_path = Path(f"{project_root}/data")
    data_folder_path.mkdir(parents=True, exist_ok=True)

    # Return the path to the folder
    return data_folder_path.absolute()

In [ ]:
from pathlib import Path

def get_export_folder_path():
    # Construct the path to the export folder and create it
    project_root = get_project_root_folder()
    export_folder_path = Path(f"{project_root}/data/reports")
    export_folder_path.mkdir(parents=True, exist_ok=True)

    # Return the path to the folder
    return export_folder_path.absolute()

In [ ]:
import re

def clean_string(string):
    return re.sub("[^0-9a-zA-Z ]+", "", string).replace(" ", "-")